<a href="https://colab.research.google.com/github/kevoniano/AI-stuff/blob/master/DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DQN con Keras y AI Gym

En este notebook vamos a usar AI Gym y Keras para implementar el algoritmo DQN. 


---

Keras es una librería de alto nivel para la construcción de redes neuronales que usa como back end Tensorflow. 

AI Gym es una librería que "empaqueta" escenarios complejos y expone unos cuántos métodos para interactuar con ellos. 

En este ejemplo usaremos [cartpole-v1](https://www.youtube.com/watch?v=0fNjV2icRd0). La tarea es usar dos acciones (izquierda y derecha) para mantener vertical una plataforma sobre la que hay un péndulo invertido. 

La [documentación en Github](https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py) muestra que el estado se conforma por 4 variables, angulo y su derivada, posicion en x y su derivada. 

El algoritmo recibirá un estado y deberá generar una salida: 0 o 1, indicando al programa hacia donde moverse. 

Por cada paso que el péndulo se mantenga cerca de la vertical **el agente recibe 1 punto**, por lo que para maximizar su recompensa debe mantenerse vertical el mayor número posible de pasos. **El máximo es 501.**

El experimento tambien se puede correr offline [clonando este repositorio](https://github.com/jonaths/dqn-diplomado). 

## Importar dependencias
Keras, AI Gym y Numpy

In [1]:
import gym
import random
import os
import sys
import numpy as np
from collections      import deque
from keras.models     import Sequential
from keras.layers     import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


## Instanciar al agente
Esta clase construye la red neuronal y contiene los métodos con los que el agente aprende y guarda el modelo entrenado. 

In [0]:
class Agent():
    def __init__(self, state_size, action_size):
        self.weight_backup      = "cartpole_weight.h5"
        self.state_size         = state_size
        self.action_size        = action_size
        # tamano del replay memory
        self.memory             = deque(maxlen=2000)
        self.learning_rate      = 0.001
        self.gamma              = 0.95
        self.exploration_rate   = 1.0
        self.exploration_min    = 0.01
        self.exploration_decay  = 0.995
        # instanciar la red neuronal al crear el modelo
        self.brain              = self._build_model()

    def _build_model(self):
        """
        Construye el modelo o lo carga desde un archivo h5
        :return:
        """
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))

        if os.path.isfile(self.weight_backup):
            model.load_weights(self.weight_backup)
            self.exploration_rate = self.exploration_min
        return model

    def save_model(self):
        # guardar el modelo
        self.brain.save(self.weight_backup)

    def act(self, state):
        """
        Recibe un estado y regresa una accion 
        :param state: un vector con shape (1, 4)
        :return:
        """
        # epsilon greedy
        if np.random.rand() <= self.exploration_rate:
            return random.randrange(self.action_size)
        # un vector con los valores Q de las dos posibles acciones
        # [[20.36317  20.821508]]
        act_values = self.brain.predict(state)
        # regresa el indice de la accion con mayor valor
        return np.argmax(act_values[0])

    def remember(self, state, action, reward, next_state, done):
        """
        Guardar una transaccion en replay memory
        :param state: las 4 variables que representan el estado en cartpole
            [[-0.02476985 -0.04408725 -0.0043659   0.03441053]]
        :param action: el indice de la accion
            0 o 1
        :param reward: la recompensa
            1.0
        :param next_state: el nuevo estado
            [[-0.02565159  0.15109704 -0.00367769 -0.25964668]]
        :param done: si el episodio termino
            True o False
        :return:
        """
        self.memory.append((state, action, reward, next_state, done))
 
    def replay(self, sample_batch_size):
        """
        Entrena sacando informacion de replay memory
        :param sample_batch_size: el tamaño del mini batch
        :return: 
        """
        # si no hay suficientes muestras no hace nada
        if len(self.memory) < sample_batch_size:
            return
        # toma una muestra aleatoria de replay memory
        sample_batch = random.sample(self.memory, sample_batch_size)
        
        # entrena la red neuronal con el mini batch
        for state, action, reward, next_state, done in sample_batch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.brain.predict(next_state)[0])
            target_f = self.brain.predict(state)
            target_f[0][action] = target
            
            # entrenar: este estado (st ate) debe tener este valor (target_f)
            # fit resta la estimacion actual, asi solo pasamos el target
            self.brain.fit(state, target_f, epochs=1, verbose=0)
            
        # actualiza epsilon
        if self.exploration_rate > self.exploration_min:
            self.exploration_rate *= self.exploration_decay

## Construir el experimento
Esta clase instancía el escenario de AI Gym y corre el experimento. 

Entradas:
- episodes: el numero de episodios (recomendado 10,000)
- batch_size: el tamaño del minibatch (recomendado 32)

In [0]:
class CartPole:
    def __init__(self):
        self.sample_batch_size = 32
        self.episodes          = 10000
        self.env               = gym.make('CartPole-v1')

        self.state_size        = self.env.observation_space.shape[0]
        self.action_size       = self.env.action_space.n
        self.agent             = Agent(self.state_size, self.action_size)


    def run(self):
        print("=== Running ")
        try:
            sample_results = []
            for index_episode in range(self.episodes):
                state = self.env.reset()
                state = np.reshape(state, [1, self.state_size])

                done = False
                index = 0
                while not done:
                    # self.env.render()

                    action = self.agent.act(state)

                    next_state, reward, done, _ = self.env.step(action)
                    next_state = np.reshape(next_state, [1, self.state_size])
                    self.agent.remember(state, action, reward, next_state, done)
                    state = next_state
                    index += 1
                    
                # descomentar para imprimir cada episodio
                # print("Episode {}# Score: {}".format(index_episode, index + 1))
                
                # para la estadistica
                sample_results.append(index + 1)
                
                # imprime cada 25 episodios
                if index_episode > 0 and index_episode % 25 == 0:
                    print("Episode {}# Avg Score: {}, Min: {}, Max: {} "
                          .format(index_episode, 
                                  sum(sample_results) / len(sample_results), 
                                  min(sample_results), max(sample_results))
                         )
                    sample_results = []
                self.agent.replay(self.sample_batch_size)
        finally:
            self.agent.save_model()

## Correr el experimento
Estos dos comandos instancian el escenario y corren el experimento. 

Despues de los 4000-5000 episodios  el programa devolverá resultados con maximos de 501 puntos. Esto quiere decir que hubo episodios donde el péndulo se mantuvo vertical por 501 episodios. 

Se puede subir un archivo de pesos cartpole_weight.h5 y el programa lo usará.

In [4]:
cartpole = CartPole()
cartpole.run()

W0629 17:44:08.565289 140465407723392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0629 17:44:08.600854 140465407723392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0629 17:44:08.611209 140465407723392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0629 17:44:08.667808 140465407723392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0629 17:44:08.688434 140465407723392 deprecation_wrappe

=== Running 
Episode 25# Avg Score: 25.346153846153847, Min: 10, Max: 54 
Episode 50# Avg Score: 17.8, Min: 9, Max: 31 
Episode 75# Avg Score: 17.0, Min: 9, Max: 37 
Episode 100# Avg Score: 18.2, Min: 11, Max: 42 
Episode 125# Avg Score: 15.04, Min: 10, Max: 32 
Episode 150# Avg Score: 22.96, Min: 10, Max: 65 
Episode 175# Avg Score: 34.08, Min: 14, Max: 65 
Episode 200# Avg Score: 16.4, Min: 10, Max: 36 
Episode 225# Avg Score: 31.4, Min: 10, Max: 78 
Episode 250# Avg Score: 29.32, Min: 10, Max: 155 
Episode 275# Avg Score: 29.96, Min: 12, Max: 99 
Episode 300# Avg Score: 25.52, Min: 9, Max: 77 
Episode 325# Avg Score: 13.96, Min: 10, Max: 48 
Episode 350# Avg Score: 28.84, Min: 12, Max: 54 
Episode 375# Avg Score: 25.4, Min: 9, Max: 89 
Episode 400# Avg Score: 37.12, Min: 9, Max: 100 
Episode 425# Avg Score: 30.6, Min: 16, Max: 47 
Episode 450# Avg Score: 76.84, Min: 23, Max: 213 
Episode 475# Avg Score: 95.8, Min: 26, Max: 205 
Episode 500# Avg Score: 129.96, Min: 49, Max: 369 
Epis

KeyboardInterrupt: ignored